In [42]:
#подключаем необходимые библиотеки 
import numpy as np #для работы с массивами
import tensorflow as tf #для работы с нейросетью
import tensorflow_datasets as tfds # для загрузки датасета
from tensorflow.keras.preprocessing.image import load_img, img_to_array #нужные нам функции
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout #готовые модели нейронных сетей 

In [20]:
train, _= tfds.load('cats_vs_dogs', split=['train[:100%]'], with_info=True, as_supervised=True)#загружаем датасет cats_vs_dogs

In [21]:
#так как в датасете ресунки разных размеров,данная функция меняет размер изображения, делим размер на 255 потому что для нейросети так гораздо легче распознавать рисунок
SIZE = 224
def resize_image(img, label):
  img = tf.cast(img, tf.float32)
  img = tf.image.resize(img, (SIZE, SIZE))
  img = img / 255.0
  return img, label

In [22]:
#тут приводим рисунки из датасета в нужный нам размер и делим его так как целиком она не поместиться в память видеокарты
train_resized = train[0].map(resize_image)
train_batches = train_resized.shuffle(1000).batch(16)

In [23]:
#берем кусок из уже натренированного неросети “keras ”, базывые ее слои, ее не надо тренировать
base_layers = tf.keras.applications.MobileNetV2(input_shape=(SIZE, SIZE, 3), include_top=False)
base_layers.trainable = False

In [24]:
#собираем нашу нейросеть
#последовательно объявляем части нашей нейросети 

model = tf.keras.Sequential([
                             base_layers,#Начальный базовый слой, которую мы выше взяли от нейросети keras
                             GlobalAveragePooling2D(),#уменьшить картинку и учредить значение разных фитч
                             Dropout(0.2),#20 % нейронов в процессе обучения случайным образом будут выключаться , это нужно чтобы нейросеть не заучивала наши данные , а что бы она пыталась найти какие то общие фичы
                             Dense(1)
])
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])#скомпилируем модель

In [25]:
#начинаем обучать нашу модель (нейросеть)
#вызываем функцию fit. указываем данные на которых будет обучаться наша модель и также количество эпох, это то сколько раз пройдется наша модель по этим данным 
model.fit(train_batches, epochs=1)
#в первой эпохе нейросеть обучилась точностью в 97,65 процентов  

1454/1454 [==============================] - 775s 529ms/step - loss: 0.0651 - accuracy: 0.9765


In [26]:
model.fit(train_batches, epochs=2)
#после второго и третьего прогона точность дошла до  98,62 процента

Epoch 1/2
1454/1454 [==============================] - 740s 507ms/step - loss: 0.0455 - accuracy: 0.9835
Epoch 2/2
1454/1454 [==============================] - 801s 550ms/step - loss: 0.0400 - accuracy: 0.9862


In [33]:
#загружаю изображение 
#приминаю на нем те же самые преобразование, чтобы это подходило для нейросети  
#и вызываю у модели функцию predict, которая и предскажет что же это, кошка или собака 

img = load_img(r'C:\Users\Zhenis\Desktop\cats_and_dogs\dog.jpg')
img_array = img_to_array(img)
img_resized,_ = resize_image(img_array,_)
img_expended = np.expand_dims(img_resized, axis =0)
model.predict(img_expended)

#на что нейросеть мне ответила значением 5.24
#в начале в датасете кочки были отмечены нулем, а собаки единицей 
#и выходит так что если значение функций predict больше 0.5 то это собака, а если меньше 0.5 то это кошка , и чем дальше ответь нейросети от 0.5 тем она уверене в том что это собака или кошка  
#у меня была фотка собаки и нейросеть более  чем уверена что это собака


1/1 [==============================] - 0s 67ms/step


array([[5.2415915]], dtype=float32)